### 네이버 카페 크롤러

In [12]:
#chrome버전과 맞는 chromedriver 설치

In [13]:
!pip install bs4
!pip install webdriver_manager
!pip install selenium==3.141.0
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.naver.com/')

In [14]:
#필요 라이브러리 import 
#selenium 셋팅
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait #크롤링 wait
from selenium.webdriver.support import expected_conditions as EC #예외상항 에러 넘김
from selenium.webdriver.chrome.options import Options # chromedriver 돌릴때 셋팅

In [15]:
import time
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

In [16]:
#네이버 로그인
#자동화된 크롬창 실행시켜 네이버 로그인 페이지 접속
#1 크롬드라이버 제어(크롬창 켜기)
#2 네이버 로그인(아이디 > 비밀번호 > 로그인 버튼 클릭 순서로 작업 진행)

In [17]:
#크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')

#현재 컴퓨터 크롬 드라이버 위치(크롬드라이버 같은 폴더에 위치하게 함)
chrome_path = 'chromedriver.exe'

#자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)

C:\Users\user\AppData\Local\Temp\ipykernel_13068\1853397687.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)


In [18]:
#네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

#본인의 아이디, 비밀번호를 각 변수에 저장
my_id='scv0206'
my_pw='DlDpWl026!@'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

#'로그인' 버튼 클릭
driver.find_element('id','log.login').click()
time.sleep(1)

#자동 로그인 등록안함
driver.find_element('id','new.dontsave').click()

In [19]:
#크롤링 할 카페 접속 및 우너하는 게시판 클릭
# 1 카페 접속
# - 로그인한 계정

In [20]:
#카페접속
time.sleep(2)
moms_url = 'https://cafe.naver.com/feko' # 타겟카페 url
driver.get(moms_url)
time.sleep(1)

# 원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink143"]').click() # 게시판의 선택자
time.sleep(1)

#프레임 전환
#iframe에 있는 내용 크롤링, 영역설정
driver.switch_to.frame('cafe_main')
time.sleep(1)

In [21]:
#게시물 크롤링
# 1 첫번째 게시물 클릭 Xpath활용
# 2 크롤링할 개수 지정
# 3 크롤링

In [22]:
# 제목, 본문, 댓글, 시간 빈 list 생성

titles = [] #제목
reviews = [] #리뷰
comments = [] #댓글
date = [] # 시간

count_view=0


# 1. 첫번째 게시물 클릭 (선택자 입력_여기서는 xpath copy 해줌)
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]').click()
#driver.find_element('xpath','//*[@id="main-area"]/div[5]/table/tbody/tr[5]/td[1]/div[2]/div/a[1]').click()


# 2. for 문으로 지정한 게시물 개수까지 돌림

for i in range(1): #새로 창열기
    for i in range(30): # 한 번에 몇 개의 개시물을 크롤링 할 것인지 지정        
        driver.implicitly_wait(30) 
        
        # 제목 수집
        time.sleep(random.uniform(0.5,1)) # 0.5~1초 랜덤한 시간으로 쉬어줘
        title = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)    

       
        # 글 본문 수집 (3가지 경우 존재)
        # 본문의 경우 선택자 1을 확인하고 코드를 작성했으나, 또 다른 선택자가 존재하여 선택자 2를 다시 try해주도록 만들었습니다.
        try: 
            content = driver.find_element(By.CSS_SELECTOR,'div.se-module.se-module-text').text # 선택자1 ('div.se-module.se-module-text')
            reviews.append(content)       
            
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR,'div.ContentRenderer').text # 선택자2 ('div.ContentRenderer')
                reviews.append(content)
            except:
                content= [] # 본문이 사진으로만 구성 (담을게 없어서 빈 리스트 생성해서 append 시켜줌)
                reviews.append(content)


         # 댓글 수집 (2가지 경우 존재)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span',class_='text_comment') # find_all = text_comment 전부 다 찾아줘
        if len(iscomment) == 0: # 댓글 없는 경우
            box = []

        else: # 댓글 있는 경우
                
            WebDriverWait(driver, 15).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "text_comment")) )
                # 웹드라이버를 통해 브라우져에서 최대 15초 기다려주고, text_comment(댓글 선택자) 기준으로 나올때 까지 기다려줘
            
            soup = bs(driver.page_source, 'lxml')  
            iscomment = soup.find_all('span',class_='text_comment') 
            box = []
            for i in iscomment:
                box.append([i.get_text()]) # iscome에 할당한 text_comment에서 text만 box에 append

        comments.append(box)

        # 시간 수집
        times = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date.append(times) 
        
        
        # count view 크롤링 진행상황 보기 (50개 마다 출력)
        count_view = count_view + 1
        if count_view % 50 ==0: #50으로 나누었을 때 0이 되면 게시물 크롤링 완료라고 보여줘
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass
        

        
        
        # 다음글 클릭 (다음글 버튼 클릭해서 연속적으로 크롤링, 첫번째글->두번째글->세번째글...)
        try:
            driver.find_element(By.CSS_SELECTOR,"#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span").click()
            driver.implicitly_wait(28)
            
        except: # 다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            
            current_url = driver.current_url # 다음글 버튼이 사라져서 크롤링이 멈춘 url 가지고 오기
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  


            ##재로그인
            # 네이버 로그인 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(9)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)


            # '로그인' 버튼 클릭
            driver.find_element('id','log.login').click()
            time.sleep(1)

            # 멈친 부분의 링크 가지고오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)    

                  
    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
    link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")
    
    #현재 크롬창 닫기
    driver.close()
    
    
    # 크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  
    
       
    ##재로그인
    # 네이버 로그인 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(9)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    # '로그인' 버튼 클릭
    driver.find_element('id','log.login').click()
    time.sleep(1)
    
    #다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')


# 크롬 창 닫기
driver.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_13068\324080789.py:120: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [23]:
data = {'time':date, 'title':titles, 'reviews':reviews,'comment':comments}
dataDF = pd.DataFrame(data)
dataDF.to_csv('여우야카페.csv',encoding='utf-8-sig')
dataDF

,time,title,reviews,comment
0,2022.11.10. 15:00,예민보스피부는 필터샤워기 필수일까요?,친구가 너같은 예민보스피부는 필터샤워기 필수라고\n집에 당장 달아두라고 이야기를 하...,[]
1,2022.11.10. 14:59,발관리에 요즘 관심이가서..,요즘 부쩍 발관리에 관심이 많이 가더라구요 손관리에 조금 권태기가 왔달까ㅎㅎ..\n...,[[발관리면 저는 문제성발톱전문 닥터페디 세미나 들어보시는거 추천이요 정말 발관리의...
2,2022.11.10. 14:56,오늘 저녁에 뭐먹을까요,막창 삼겹살 곱창전골 닭갈비 중에서 골라주세여!!,"[[곱창전골!], [제 최애 ㅎㅎㅎㅎㅎㅎㅎ], [와 맛있는 후보들이네여 ㅋㅋㅋ 저는..."
3,2022.11.10. 14:55,날씨는 좋은데 미세먼지가ㅠㅠㅠㅠㅠ,아 숨이 턱턱 막혀요 마스크도 못벗겠구ㅠㅠㅠㅠㅠㅠㅠㅠㅠ,[[하늘이 뿌옇더라니 ㅠㅠㅠㅠㅠㅠㅠ]]
4,2022.11.10. 14:54,전장연 지하철시위 장애인 지하철시위 개인적인 생각..,솔직히 그냥 남일이라고 생각했고\n출근하는분들 고생많으시겠다 했는데\n제가 주로 이...,"[[진짜 너무해요 대체 언제까지 하는건지], [진짜 너무 심하다생각해요..], [반..."
5,2022.11.10. 14:53,시간이 약이겟죠,ㅠㅠ 아직 한달도 안되서 그런가 붓기가~~~ㅜㅜ,[[무슨 수술하신거에요???]]
6,2022.11.10. 14:52,어제도 지옥철이라서 밥먹고 지하철 타러 갔는데..,어제 마라탕 먹고 아 퇴근시간 지났으니깐 사람 없겠다 하고 지하철 타러 갔는데 그래...,[]
7,2022.11.10. 14:52,입술 필러는 어디가 잘해요??,맛집 추천좀해주세요 ㅎㅎ 서울에서!,"[[지역 서울이면 다 상관없어요??], [입술필러 뷰티바 어린공주 여기 많이 가는것..."
8,2022.11.10. 14:51,대패 집에 비빔면 팔 생각 한 분은 천재가 분명해요...,진짜 뭘 좀 아시네요,"[[먹잘알.....], [쩝쩝박사임], [그니까요 처음에 누가 생각한건지 굿], [..."
9,2022.11.10. 14:51,오늘,날씨포근하네요,[]
